In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [3]:
# Define the number of transactions
num_transactions = 1000

# Generate sample data for the travel_details dataset
vendor_ids = np.random.choice(np.arange(1000, 1016), size=num_transactions, replace=True)
transport_types = np.random.choice(['Roadways', 'Railways', 'Waterways'], size=num_transactions)
transaction_dates = [datetime(2022, np.random.randint(1, 13), np.random.randint(1, 29)) for _ in range(num_transactions)]
goods_types = np.random.choice(['Oil', 'Coal', 'Natural Gas'], size=num_transactions)
committed_quantity = np.random.randint(1000, 5000, size=num_transactions)
committed_timeline = np.random.randint(1, 10, size=num_transactions)
committed_cost = np.random.randint(10000000, 50000000, size=num_transactions)
actual_timeline = committed_timeline + np.random.randint(1, 5, size=num_transactions)
actual_cost = committed_cost + np.random.randint(100000, 500000, size=num_transactions)
actual_quantity = np.random.randint(0.8 * committed_quantity, committed_quantity, size=num_transactions)


In [4]:
# Ensure actual quantity is lesser than committed quantity
actual_quantity = np.minimum(actual_quantity, committed_quantity)

# Ensure actual timeline is greater than committed timeline
actual_timeline = np.maximum(actual_timeline, committed_timeline)

# Ensure actual cost is greater than committed cost
actual_cost = np.maximum(actual_cost, committed_cost)

In [5]:
# Create DataFrame for travel_details dataset
travel_details = pd.DataFrame({
    'Goods': goods_types,
    'Committed Quantity': committed_quantity,
    'Committed Timeline': committed_timeline,
    'Committed Cost': committed_cost,
    'Actual Timeline': actual_timeline,
    'Actual Cost': actual_cost,
    'Actual Quantity': actual_quantity,
    'Transaction Date': transaction_dates,
    'Transport Type': transport_types,
    'Vendor ID': vendor_ids
})

In [6]:
# Derive delay from actual and committed timelines
travel_details['Delay'] = travel_details['Actual Timeline'] - travel_details['Committed Timeline']

# Derive month and quarter from the transaction date
travel_details['Month'] = travel_details['Transaction Date'].dt.month
travel_details['Quarter'] = travel_details['Transaction Date'].dt.quarter

In [7]:
travel_details['Committed Cost']/=10**7
travel_details['Actual Cost']/=10**7

In [8]:
travel_details.to_csv('final_transport.csv', index=False)

In [9]:
# Define the number of vendors
num_vendors = 15

# Generate sample data for the vendor dataset
vendor_data = {
    'Vendor ID': np.arange(1001, 1016),
    'Vendor Name': ['Vendor' + str(i) for i in range(1, num_vendors + 1)],
    'Year of Establishment': np.random.randint(1950, 2004, size=num_vendors),
    'Total Volume': np.random.randint(300000, 800000, size=num_vendors),
    'Days for Payment Clearance': np.random.choice(np.arange(15, 61, step=15), size=num_vendors),
    'Consistency': np.random.randint(1, 5, size=num_vendors),  # Assuming a scale of 1 to 5
    'Number of Sources': np.random.randint(25, 100, size=num_vendors)
}

# Create DataFrame for vendor dataset
vendor_df = pd.DataFrame(vendor_data)


In [10]:
vendor_df['Experience']=2024-vendor_df['Year of Establishment']

In [11]:
# Compute the sum of actual quantity by vendor ID in travel_details
actual_quantity_sum_by_vendor = travel_details.groupby('Vendor ID')['Actual Quantity'].sum().reset_index()

# Merge the sum of actual quantity with vendor_df
vendor_df = pd.merge(vendor_df, actual_quantity_sum_by_vendor, on='Vendor ID', how='left')

# Rename the column to indicate it represents the sum of actual quantity
vendor_df.rename(columns={'Actual Quantity': 'Total Actual Quantity'}, inplace=True)

In [12]:
vendor_df['Remaining Volume']=vendor_df['Total Volume']-vendor_df['Total Actual Quantity']

In [14]:
actual_quantity_sum_by_vendor = travel_details.groupby('Vendor ID')['Actual Cost'].sum().reset_index()
vendor_df = pd.merge(vendor_df, actual_quantity_sum_by_vendor, on='Vendor ID', how='left')
vendor_df.rename(columns={'Actual Cost': 'Total Revenue'}, inplace=True)

In [15]:
vendor_df.to_csv('vendor.csv',index=False)